# Notebook para la extracción de datos

Notebook donde se realiza la extracción de datos a través de la API Ergast y su posterior guardado en CSVs para facilitar la lectura de datos en el proyecto.

In [ ]:
import requests
import csv
import pandas as pd
import json
import ast

#### Función para guardar los datos en formato CSV

In [ ]:
def create_csv(data_list, filename):
    all_keys = set()
    for item in data_list:
        all_keys.update(item.keys())
    with open(f'../data/{filename}', 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, all_keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

#### Circuitos

In [ ]:

def get_circuits(limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/circuits.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("CircuitTable", {}).get("Circuits", [])

all_results = []


limit = 100  # Num of results per page
offset = 0  # Results init

while True:
    results_season = get_circuits(limit, offset)
    if not results_season:
        break

    all_results.extend(results_season)
    offset += limit

create_csv(all_results, 'circuits.csv')


#### Pilotos [2000-2024]

In [ ]:

def get_drivers(season, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/drivers.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("DriverTable", {}).get("Drivers", [])

season_range = range(2000,2025)
all_results = []

for season in season_range:
    limit = 100  # Num of results per page
    offset = 0  # Results init

    while True:
        results_season = get_drivers(season, limit, offset)
        if not results_season:
            break

        all_results.extend(results_season)        
        offset += limit

create_csv(all_results, 'drivers_2000-2024.csv')


#### Grandes premios [2000-2024]

In [ ]:

def get_races(season, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/races.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

season_range = range(2000,2025)
all_results = []

for season in season_range:
    limit = 100  # Num of results per page
    offset = 0  # Results init

    while True:
        results_season = get_races(season, limit, offset)
        if not results_season:
            break

        all_results.extend(results_season)
        offset += limit

create_csv(all_results, 'races_2000-2024.csv')


#### Resultados carreras [2000-2024]

In [ ]:

def get_results(season, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/results.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

season_range = range(2000,2025)
all_results = []

for season in season_range:
    limit = 100  # Num of results per page
    offset = 0  # Results init

    while True:
        results_season = get_results(season, limit, offset)
        if not results_season:
            break

        all_results.extend(results_season)
        
        offset += limit

create_csv(all_results, 'results_2000-2024.csv')


#### Resultados carreras formato Sprint
Formato disponible a partir de 2021

In [ ]:
def get_results(season, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/sprint.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

season_range = range(2000,2025)
all_sprint_results = []

for season in season_range:
    limit = 100  # Num of results per page
    offset = 0  # Results init

    while True:
        sprint_results_season = get_results(season, limit, offset)
        if not sprint_results_season:
            break

        all_sprint_results.extend(sprint_results_season)
        
        offset += limit

create_csv(all_sprint_results, 'sprint_results_2000-2024.csv')

#### Grandes premios por temporada [2000-2024]

In [ ]:

def get_seasons(season, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

season_range = range(2000,2025)
all_results = []

for season in season_range:
    limit = 100  # Num of results per page
    offset = 0  # Results init

    while True:
        circuits_season = get_seasons(season, limit, offset)
        if not circuits_season:
            break

        all_results.extend(circuits_season)        
        offset += limit

create_csv(all_results, 'seasons_circuits_2000-2024.csv')


#### Pit stops
Tiempos disponibles a partir de 2011

In [ ]:
season_range = range(2000,2025)

all_pitstops = []
def get_pitstops(season, round_number, limit, offset):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/{round_number}/pitstops.json', params={"limit": limit, "offset": offset})
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

for season in season_range:
    season_rounds_resp = requests.get(f'https://ergast.com/api/f1/{season}.json')

    season_rounds = season_rounds_resp.json()

    rounds_list = season_rounds.get("MRData", {}).get("RaceTable", {}).get("Races", [])
    for round_dict in rounds_list:
        limit = 100  # Num of results per page
        offset = 0  # Results init
        while True:
            round_number = round_dict["round"]
            pitstops = get_pitstops(season, round_number, limit, offset)
            if not pitstops:
                break
            
            all_pitstops.extend(pitstops)
            offset += limit
        
create_csv(all_pitstops, 'raw_pitstops.csv')

#### Resultados clasificaciones
Los resultados de las clasificaciones están disponibles a partir de 2023

In [ ]:
season_range = range(2003,2025)

def get_qualifying_results(season, round):
    results_response = requests.get(f'https://ergast.com/api/f1/{season}/{round}/qualifying.json')
    results = results_response.json()
    return results.get("MRData", {}).get("RaceTable", {}).get("Races", [])

rounds_per_season_df = pd.read_csv('../data/races_2000-2024.csv')
all_qualifying_results = []
for season in season_range:
    rounds = rounds_per_season_df[rounds_per_season_df['season'] == season]['round']
    rounds_list = rounds.tolist()

    for n_round in rounds_list:
        res = get_qualifying_results(season, n_round)
        all_qualifying_results.extend(res)


create_csv(all_qualifying_results, 'qualifying_results_2003-2024.csv')